In [8]:
import torch
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

In [51]:
df_train = pd.read_csv("/content/drive/MyDrive/Sign_Language/sign_mnist_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Sign_Language/sign_mnist_test.csv")

In [10]:
# Separate labels and pixel values
labels = df_train.iloc[:, 0].values
pixels = df_train.iloc[:, 1:].values
print(f"labels = {labels}\npixels = {pixels}")

labels = [ 3  6  2 ... 18 17 23]
pixels = [[107 118 127 ... 204 203 202]
 [155 157 156 ... 103 135 149]
 [187 188 188 ... 195 194 195]
 ...
 [174 174 174 ... 202 200 200]
 [177 181 184 ...  64  87  93]
 [179 180 180 ... 205 209 215]]


In [12]:
class AlexNet(nn.Module):
  def __init__(self, num_classes, dropout=0.5):
      super(AlexNet, self).__init__()
      self.features = nn.Sequential(
          nn.Conv2d(1, 64, 5, stride=1, padding=2),  # Adjusted kernel size, stride, and padding
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),  # Adjusted kernel size and stride
          nn.Conv2d(64, 192, 5, stride=1, padding=2),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),
          nn.Conv2d(192, 384, 3, stride=1, padding=1),
          nn.ReLU(),
          nn.Conv2d(384, 256, 3, stride=1, padding=1),
          nn.ReLU(),
          nn.Conv2d(256, 256, 3, stride=1, padding=1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2)  # Adjusted kernel size and stride
      )
      self.classifier = nn.Sequential(
          nn.Dropout(p=dropout),
          nn.Linear(256 * 3 * 3, 4096),  # Adjusted dimensions based on feature map size
          nn.ReLU(),
          nn.Dropout(p=dropout),
          nn.Linear(4096, 4096),
          nn.ReLU(),
          nn.Linear(4096, num_classes),  # Adjusted to match the number of classes
          nn.Softmax(dim=1)
      )

  def forward(self, x):
      x = self.features(x)
      x = x.view(x.size(0), -1)  # Flatten the tensor
      x = self.classifier(x)
      return x

In [30]:
num_classes = df_train.iloc[:, 0].nunique() # starts from 1
print(f'Number of unique labels: {num_classes}')

Number of unique labels: 24


In [68]:
# Define your device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AlexNet(num_classes=num_classes).to(device)
epochs = 100 # number of single passes on the network
loss_fnPrecents = nn.CrossEntropyLoss(ignore_index=24)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [45]:
class SignDataset(Dataset):
    def __init__(self, pixels, labels, transform=None):
        self.pixels = pixels
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.pixels[idx].reshape(28, 28).astype('float32')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [69]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = SignDataset(pixels, labels, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=100)

In [70]:
# Training loop
def train_model(model, dataloader, epochs, optimizer, loss_fn, device):
    model.train()
    for epoch in range(epochs):
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = loss_fn(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

def evaluate_model(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    total, correct = 0, 0
    with torch.no_grad():
        for data, targets in dataloader:
            data, targets = data.to(device), targets.to(device)  # Ensure data and targets are on the correct device
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    return 100 * correct / total  # Return accuracy as a percentage

In [ ]:
train_model(model=model,
            dataloader=train_dataloader,
            epochs=epochs,
            optimizer=optimizer,
            loss_fn=loss_fnPrecents,
            device=device)

In [57]:
test_dataset = SignDataset(pixels, labels, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=100)

In [66]:
accuracy_train = evaluate_model(
    model=model,
    dataloader=train_dataloader,
    device=device)
print(f"train accuracy = {accuracy_train}")

train accuracy = 3.0


In [67]:
accuracy_test = evaluate_model(
    model=model,
    dataloader=test_dataloader,
    device=device)
print(f"test accuracy = {accuracy_test}")

test accuracy = 3.0
